---
title: "Creating AI character - interactive two player prompt duel game 🤐"
author: "Jaekang Lee"
date: "2024-06-31"
categories: [llm, prompt_engineering, research, python]
description: Teaching AI to keep a character vs gaslighting AI to break its character
toc: true
execute:
    echo: false
    warning: false
---

In [43]:
from dotenv import load_dotenv
import os


api_key=os.getenv("OPENAI_API_KEY")
import os
from openai import AzureOpenAI
print(api_key)

#api_key = "bac09550d21e4536b3aef7006af9d1bb"
#api_key = os.environ['OPENAI_API_KEY']
azure_endpoint = "https://azure-open-ai-common-infra.openai.azure.com/"
api_version = "2023-07-01-preview"
deployment_name='ailab-llm' 

client = AzureOpenAI(
        api_key=api_key,  
        api_version=api_version,
        azure_endpoint = azure_endpoint
    )
    

    
# Send a completion call to generate an answer
print('Sending a test completion job')
start_phrase = 'Write a tagline for an ice cream shop. '
response = client.chat.completions.create(
    model=deployment_name,
                messages=[
                    {"role": "system", "content": f"You are an AI assistant. Respond to the user's message"}], max_tokens=10)
    

bac09550d21e4536b3aef7006af9d1bb
Sending a test completion job


In [36]:
import os
from openai import AzureOpenAI

import ipywidgets as widgets
from IPython.display import display, clear_output

class AIGame:
    def __init__(self, api_key, azure_endpoint, api_version, deployment_name):
        self.api_key = api_key
        self.azure_endpoint = azure_endpoint
        self.api_version = api_version
        self.deployment_name = deployment_name
        self.client = AzureOpenAI(api_key = self.api_key,  
            api_version=self.api_version,
            azure_endpoint = self.azure_endpoint)
        self.taboo_word = ""
        self.player1_prompt = ""
        self.player2_prompt = ""
        self.ai_response = ""
        self.current_player = 1
        self.game_over = False
        
        # Create widgets
        self.word_input = widgets.Text(description="Taboo Word:")
        self.prompt_input = widgets.Textarea(description="Prompt:", rows=3)
        self.submit_button = widgets.Button(description="Submit")
        self.output = widgets.Output()
        
        # Set up button click event
        self.submit_button.on_click(self.on_submit)
        
    def display_game(self):
        display(self.word_input, self.prompt_input, self.submit_button, self.output)
        
    def on_submit(self, b):
        with self.output:
            clear_output()
            if self.current_player == 1:
                self.taboo_word = self.word_input.value.lower()
                self.player1_prompt = self.prompt_input.value
                print(f"Player 1 has set the taboo word and created a prompt.")
                self.current_player = 2
                self.word_input.disabled = True
            elif self.current_player == 2:
                self.player2_prompt = self.prompt_input.value
                self.get_ai_response()
                self.game_over = True
                self.display_results()
            
            if not self.game_over:
                print(f"Player {self.current_player}'s turn.")
            
            self.prompt_input.value = ""
    
    def get_ai_response(self):
        try:
            response = self.client.chat.completions.create(
                model=self.deployment_name,
                messages=[
                    {"role": "system", "content": f"You are an AI assistant. Respond to the user's message without using the word '{self.taboo_word}'."},
                    {"role": "user", "content": self.player1_prompt},
                    {"role": "assistant", "content": "Understood, I'll respond without using that word."},
                    {"role": "user", "content": self.player2_prompt}
                ]
            )
            self.ai_response = response.choices[0].message.content
        except Exception as e:
            self.ai_response = f"Error: {str(e)}"
        
    def display_results(self):
        print("Game Over!")
        print(f"Taboo word: {self.taboo_word}")
        print(f"Player 1's prompt: {self.player1_prompt}")
        print(f"Player 2's prompt: {self.player2_prompt}")
        print(f"AI response: {self.ai_response}")
        if self.taboo_word in self.ai_response.lower():
            print("Player 2 wins! The AI used the taboo word.")
        else:
            print("Player 1 wins! The AI did not use the taboo word.")

# Create and display the game
api_key = "bac09550d21e4536b3aef7006af9d1bb"
azure_endpoint = "https://azure-open-ai-common-infra.openai.azure.com/"
api_version = "2023-07-01-preview"
deployment_name='ailab-llm' 

game = AIGame(api_key, azure_endpoint, api_version, deployment_name)
game.display_game()

Text(value='', description='Taboo Word:')

Textarea(value='', description='Prompt:', rows=3)

Button(description='Submit', style=ButtonStyle())

Output()